In [ ]:
# https://jeremykun.com/2015/12/28/load-balancing-and-the-power-of-hashing/
# various type
# https://softwareengineering.stackexchange.com/questions/49550/which-hashing-algorithm-is-best-for-uniqueness-and-speed

In [43]:
class hash_antulov():
    def __init__(self, arr_size = 23):
        self.size = arr_size
        # the arreys could be alocated at moment one of them is needed
        self.bucket_arr = [None] * self.size
        self.bucket_arr2 = [None] * self.size   
        self.load1 = 0
        self.load2 = 0
        self.base_e =[1096.633,403.428,148.413, 54.598, 20.085,7.389, 2.718, 1.0 ]
        if self.size < 8: print('Size should be grater then ',self.size)     
           
    def get_hash(self, key):
        sum = 0
        for i,char in enumerate(str(key)):
            sum += ord(char) * self.base_e[i%8]
        hash_key = sum * sum    
        # The hash functions could be different, whichever is more appropriate.
        index1 = (int(sum + hash_key/100))%self.size
        index2 = (int(sum + 1000*hash_key))%self.size
        return index1,index2
            
    def add(self, new_key, new_value):
        index, index2 = self.get_hash(new_key)
        new_key_value_pair = [new_key, new_value, index, index2,0]
        # first check for update distinguished from double hash but could be used double hash
        if self.bucket_arr[index] is not None:
            if self.bucket_arr[index][0][0]== new_key:       
                self.bucket_arr[index][0][1] = new_value              
                return True
      # chaining is used for clarity and to distinguished from double hash but could be used double hash       
        if self.bucket_arr2[index2] is not None:  
            for set_of_values in self.bucket_arr2[index2]:
                if set_of_values[0] == new_key: 
                    set_of_values[1]= new_value
                    return True
         #   nothing was for updated, must will be inserted  
        if self.bucket_arr[index] is None:        
            self.bucket_arr[index] = [new_key_value_pair]  
            self.load1 += 1
            return True                   
        elif self.bucket_arr2[index2] is None:
            self.bucket_arr2[index2] = [new_key_value_pair]
            self.load2 += 1
            return True
        else:   # chaining is used to distigushed from double hash but could be used double hash
            new_key_value_pair[4] = len(self.bucket_arr2[index2])
            self.bucket_arr2[index2].append(new_key_value_pair) 
            self.load2 += 1
            return True
        return False
   
    def delete(self, key):
        index, index2 = self.get_hash(key)     
        if self.bucket_arr[index] is not None:
            if self.bucket_arr[index][0][0] == key:
                self.bucket_arr[index]= None
                self.load1 -= 1
                return True  
        if self.bucket_arr2[index2] is not None:
            for indey in range (0, len(self.bucket_arr2[index2])):
                if self.bucket_arr2[index2][indey][0] == key:
                    self.bucket_arr2[index2].pop(indey)
                    self.load2 -= 1
                    return True       
        return False    
            
    def get_value(self, key):
        index, index2 = self.get_hash(key)          
        if self.bucket_arr[index] is not None or self.bucket_arr2[index2] is not None:
            if self.bucket_arr[index][0][0] == key:
                return self.bucket_arr[index][0][1]
            for set_of_values in self.bucket_arr2[index2]:
                if set_of_values[0] == key: 
                    return set_of_values[1]
        else:  #self.bucket_arr[index] is None and self.bucket_arr2[index2] is None:
            return None   
     
    def get_loads(self):
        if self.load2 == 0:
            print('Load ratio not defined  load1={0} / load2={1}'.format(self.load1,self.load2))
        else:  # load ratio is the double ratio of load factors load1=n1/m1 over load2=n2/m2, where m1=m2  
            load_ratio = self.load1/self.load2      # load1=n1/size over load2=n2/size, load_ratio=n1/n2 
            sum_load_ratio =(self.load1+self.load2/(self.size+self.size))
            print('Load ratio load1={0} / load2={1}'.format(self.load1,self.load2))
            print('Load ratio ={0} ukupni load={1}'.format(load_ratio,sum_load_ratio ))       
        print('')
        print('load1={}  load2={} load1/size={:.2f}% load1/size={:.2f}%'
              .format(self.load1, self.load2, 100*self.load1/self.size, self.load2/self.size*100))
        
    def pr_index(self,new_key):
        index, index2 = self.get_hash(new_key) 
        print(' {} index  {}  {}   load1 = {} load2 = {} '
              .format(new_key, index, index2, self.load1, self.load2))

    def print12(self):
        print('----IMENIK TEST ----')
        Binstr0='None'
        Binstr1='None'
        for index, arr_bins in enumerate(zip(self.bucket_arr, self.bucket_arr2)):
            bins0,bins1='None', 'None'            
            if arr_bins[0] != None: bins0 = str(arr_bins[0])
            if arr_bins[1] != None: bins1 = str(arr_bins[1])
            print(f"{str(index):<3}{bins0:<51}{bins1}") 
            
#     def print22(self):  
#         print('----IMENIK TEST ----')
#         for index, bucket in enumerate(zip(self.bucket_arr, self.bucket_arr2)):
#             print(index, " ",bucket[0],"  \t"*3,bucket[1]) 
            
    def print(self):
        print('----IMENIK TEST ----')
        for index, bucket in enumerate(self.bucket_arr): print(index, bucket) 
                
    def print2(self):  
        print('----IMENIK TEST ----')
        for index, bucket in enumerate(self.bucket_arr2): print(index, bucket)  
        

In [44]:
h=hash_antulov()

In [45]:
h.add('Ema2', 'drugi')
h.pr_index('Ema2')
h.add('Bruno', ' Antulov-Fantulin')
h.pr_index('Bruno')
h.get_loads()
print('Bruno',h.get_value('Bruno'))
h.print12()

 Ema2 index  20  17   load1 = 1 load2 = 0 
 Bruno index  9  5   load1 = 2 load2 = 0 
Load ratio not defined  load1=2 / load2=0

load1=2  load2=0 load1/size=8.70% load1/size=0.00%
Bruno  Antulov-Fantulin
----IMENIK TEST ----
0  None                                               None
1  None                                               None
2  None                                               None
3  None                                               None
4  None                                               None
5  None                                               None
6  None                                               None
7  None                                               None
8  None                                               None
9  [['Bruno', ' Antulov-Fantulin', 9, 5, 0]]          None
10 None                                               None
11 None                                               None
12 None                                               None
13 None  

In [46]:
h.pr_index('Bruno Antu')

 Bruno Antu index  16  8   load1 = 2 load2 = 0 


In [47]:
h.add('Boban', '222-8888')
h.add('Ema', '099-prvi')
h.add('Klara', '293-6753 prvi')
h.add('Klara', '333-update')
h.add('Ankica', '293-8625')
h.add('Ana Marija', '852-655prvi1')
h.add('Ankica', '632-4123')
h.add('Mirko', '567-2188')
h.add('Anita', '777-8888 update')
h.add('Ema', '095-uup-drugi')
h.add('Darko', '098-967-1688')
h.add('Ema', 'nnn-yyy-update')
h.add('Bruno Antulov Fantulin', 'Antulov ')
h.print12()

----IMENIK TEST ----
0  [['Boban', '222-8888', 0, 0, 0]]                   None
1  None                                               None
2  None                                               None
3  None                                               None
4  [['Darko', '098-967-1688', 4, 21, 0]]              None
5  None                                               None
6  None                                               None
7  [['Anita', '777-8888 update', 7, 5, 0]]            None
8  None                                               None
9  [['Bruno', ' Antulov-Fantulin', 9, 5, 0]]          None
10 None                                               None
11 None                                               None
12 [['Ana Marija', '852-655prvi1', 12, 5, 0]]         None
13 [['Ema', 'nnn-yyy-update', 13, 8, 0]]              None
14 None                                               None
15 [['Ankica', '632-4123', 15, 16, 0]]                None
16 None                            

In [48]:
h.delete('Bruno')
h.print12()

----IMENIK TEST ----
0  [['Boban', '222-8888', 0, 0, 0]]                   None
1  None                                               None
2  None                                               None
3  None                                               None
4  [['Darko', '098-967-1688', 4, 21, 0]]              None
5  None                                               None
6  None                                               None
7  [['Anita', '777-8888 update', 7, 5, 0]]            None
8  None                                               None
9  None                                               None
10 None                                               None
11 None                                               None
12 [['Ana Marija', '852-655prvi1', 12, 5, 0]]         None
13 [['Ema', 'nnn-yyy-update', 13, 8, 0]]              None
14 None                                               None
15 [['Ankica', '632-4123', 15, 16, 0]]                None
16 None                            

In [25]:
class hash_antulov2(hash_antulov):
    def load_check(self):        # this function could be more sofisticated
        if self.load2 <= int(self.size/2) - 1:
            return True
        return False
    
    def add(self, new_key, new_value):
        index, index2 = self.get_hash(new_key)
#         new_key_value_pair = [new_key, new_value, index, index2,0]
        if self.load_check() == True:
            self.add_2(index, index2, new_key, new_value)
            return True
        self.add_12(index, index2, new_key, new_value)
        return True
            
    def add_2(self,index, index2,new_key, new_value):
        new_key_value_pair = [new_key, new_value, index, index2,0]
        if self.bucket_arr2[index2] is not None:
            for set_of_values in self.bucket_arr2[index2]:
                if set_of_values[0] == new_key: 
                    set_of_values[1]= new_value
                    return True
         #   nothing was for updated, will be inserted                    
        if self.bucket_arr2[index2] is None:
            self.bucket_arr2[index2] = [new_key_value_pair]
            self.load2 += 1
            return True
        else:
            new_key_value_pair[4] = len(self.bucket_arr2[index2])
            self.bucket_arr2[index2].append(new_key_value_pair) 
            self.load2 += 1
            return True
        return False
        
    def add_12(self,index, index2,new_key, new_value):
        new_key_value_pair = [new_key, new_value, index, index2,0]
        if self.bucket_arr[index] is not None: 
            if self.bucket_arr[index][0][0]== new_key:       # napravljaen uppdate return
                self.bucket_arr[index][0][1] = new_value              
                return True
        if self.bucket_arr2[index2] is not None:
            for set_of_values in self.bucket_arr2[index2]:
                if set_of_values[0] == new_key: 
                    set_of_values[1]= new_value
                    return True
         #   nothing was for updated, must will be inserted  
        if self.bucket_arr[index] is None:        
            self.bucket_arr[index] = [new_key_value_pair]  # upisuje se u arr1
            self.load1 += 1
            return True                   
        elif self.bucket_arr2[index2] is None:
            self.bucket_arr2[index2] = [new_key_value_pair]
            self.load2 += 1
            return True
        else:
            new_key_value_pair[4] = len(self.bucket_arr2[index2])
            self.bucket_arr2[index2].append(new_key_value_pair) 
            self.load2 += 1
            return True
        return False

In [26]:
h2=hash_antulov2(11)

In [27]:
h2.add('Ema2', 'drugi')
h2.pr_index('Ema2')
h2.add('Bruno', ' Antulov-Fantulin')
h2.pr_index('Bruno')
h2.get_loads()
h2.print12()

 Ema2 index  10  5   load1 = 0 load2 = 1 
 Bruno index  6  0   load1 = 0 load2 = 2 
Load ratio load1=0 / load2=2
Load ratio =0.0 ukupni load=0.09090909090909091

load1=0  load2=2 load1/size=0.00% load1/size=18.18%
----IMENIK TEST ----
0  None                                              [['Bruno', ' Antulov-Fantulin', 6, 0, 0]]
1  None                                              None
2  None                                              None
3  None                                              None
4  None                                              None
5  None                                              [['Ema2', 'drugi', 10, 5, 0]]
6  None                                              None
7  None                                              None
8  None                                              None
9  None                                              None
10 None                                              None


In [28]:
h2.pr_index('Boban')

 Boban index  7  10   load1 = 0 load2 = 2 


In [29]:
h2.add('Boban', '222-8888')
h2.add('Ema', '099-prvi')
h2.add('Klara', '293-6753 prvi')
h2.pr_index('Klara')
h2.add('Klara', '333-/188')
h2.add('Anita', '777-8888 update')
h2.add('Ema', '095-uup-drugi')
h2.add('Darko', '098-967-1688')
h2.add('Ema', 'nnn-yyy-update')
h2.add('Bruno Antulov Fantulin', 'Antulov ')
h2.print12()

 Klara index  2  3   load1 = 0 load2 = 5 
----IMENIK TEST ----
0  None                                              [['Bruno', ' Antulov-Fantulin', 6, 0, 0]]
1  [['Bruno Antulov Fantulin', 'Antulov ', 1, 9, 0]] None
2  None                                              None
3  None                                              [['Klara', '333-/188', 2, 3, 0]]
4  None                                              None
5  [['Darko', '098-967-1688', 5, 4, 0]]              [['Ema2', 'drugi', 10, 5, 0], ['Ema', 'nnn-yyy-update', 4, 5, 1]]
6  None                                              None
7  None                                              None
8  None                                              None
9  [['Anita', '777-8888 update', 9, 7, 0]]           None
10 None                                              [['Boban', '222-8888', 7, 10, 0]]


In [30]:
h2.delete('Bruno')
h2.print12()

----IMENIK TEST ----
0  None                                              []
1  [['Bruno Antulov Fantulin', 'Antulov ', 1, 9, 0]] None
2  None                                              None
3  None                                              [['Klara', '333-/188', 2, 3, 0]]
4  None                                              None
5  [['Darko', '098-967-1688', 5, 4, 0]]              [['Ema2', 'drugi', 10, 5, 0], ['Ema', 'nnn-yyy-update', 4, 5, 1]]
6  None                                              None
7  None                                              None
8  None                                              None
9  [['Anita', '777-8888 update', 9, 7, 0]]           None
10 None                                              [['Boban', '222-8888', 7, 10, 0]]


In [31]:
h2.delete('Boban')
h2.print12()

----IMENIK TEST ----
0  None                                              []
1  [['Bruno Antulov Fantulin', 'Antulov ', 1, 9, 0]] None
2  None                                              None
3  None                                              [['Klara', '333-/188', 2, 3, 0]]
4  None                                              None
5  [['Darko', '098-967-1688', 5, 4, 0]]              [['Ema2', 'drugi', 10, 5, 0], ['Ema', 'nnn-yyy-update', 4, 5, 1]]
6  None                                              None
7  None                                              None
8  None                                              None
9  [['Anita', '777-8888 update', 9, 7, 0]]           None
10 None                                              []


In [32]:
h2.add('Boban', '222-8888')

True

In [33]:
h2.pr_index('Boban')
h2.get_loads()
h2.print12()

 Boban index  7  10   load1 = 3 load2 = 4 
Load ratio load1=3 / load2=4
Load ratio =0.75 ukupni load=3.1818181818181817

load1=3  load2=4 load1/size=27.27% load1/size=36.36%
----IMENIK TEST ----
0  None                                              []
1  [['Bruno Antulov Fantulin', 'Antulov ', 1, 9, 0]] None
2  None                                              None
3  None                                              [['Klara', '333-/188', 2, 3, 0]]
4  None                                              None
5  [['Darko', '098-967-1688', 5, 4, 0]]              [['Ema2', 'drugi', 10, 5, 0], ['Ema', 'nnn-yyy-update', 4, 5, 1]]
6  None                                              None
7  None                                              None
8  None                                              None
9  [['Anita', '777-8888 update', 9, 7, 0]]           None
10 None                                              [['Boban', '222-8888', 7, 10, 0]]


In [34]:
h2.add('Bruno', 'Ant-Fant')
h2.pr_index('Bruno')
h2.get_loads()
h2.print12()

 Bruno index  6  0   load1 = 3 load2 = 5 
Load ratio load1=3 / load2=5
Load ratio =0.6 ukupni load=3.227272727272727

load1=3  load2=5 load1/size=27.27% load1/size=45.45%
----IMENIK TEST ----
0  None                                              [['Bruno', 'Ant-Fant', 6, 0, 0]]
1  [['Bruno Antulov Fantulin', 'Antulov ', 1, 9, 0]] None
2  None                                              None
3  None                                              [['Klara', '333-/188', 2, 3, 0]]
4  None                                              None
5  [['Darko', '098-967-1688', 5, 4, 0]]              [['Ema2', 'drugi', 10, 5, 0], ['Ema', 'nnn-yyy-update', 4, 5, 1]]
6  None                                              None
7  None                                              None
8  None                                              None
9  [['Anita', '777-8888 update', 9, 7, 0]]           None
10 None                                              [['Boban', '222-8888', 7, 10, 0]]
